In [1]:
# Face Recognition Model Code

from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
from torchvision import datasets
from torch.utils.data import DataLoader
from PIL import Image
import os
import csv
from datetime import datetime

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
mtcnn = MTCNN(image_size=240, margin=0, min_face_size=20)
resnet = InceptionResnetV1(pretrained='vggface2').eval()

In [3]:
dataset = datasets.ImageFolder("../Images/Class/")
idx_to_class = {i: c for c, i in dataset.class_to_idx.items()}

In [4]:
def collate_fn(x):
    return x[0]

loader = DataLoader(dataset, collate_fn=collate_fn)

face_list = []
name_list = []
embedding_list = []

for img, idx in loader:
    face, prob = mtcnn(img, return_prob=True)
    if face is not None and prob > 0.90:
        emb = resnet(face.unsqueeze(0))
        embedding_list.append(emb.detach())
        name_list.append(idx_to_class[idx])


[W NNPACK.cpp:53] Could not initialize NNPACK! Reason: Unsupported hardware.


In [5]:
data = [embedding_list, name_list]
torch.save(data, 'data.pt')

In [6]:
def get_current_datetime():
    now = datetime.now()
    date = now.strftime("%Y-%m-%d")
    time = now.strftime("%H:%M:%S")
    day = now.strftime("%A")
    return date, time, day

In [10]:
def predict_and_record(folder_path, data_path, output_csv):
    with open(output_csv, 'w', newline='') as csvfile:
        fieldnames = ['Predicted Name', 'Date', 'Time', 'Day']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        saved_data = torch.load(data_path)
        embedding_list = saved_data[0]
        name_list = saved_data[1]

        for filename in os.listdir(folder_path):
            if filename.endswith(".png"): 
                img_path = os.path.join(folder_path, filename)
                img = Image.open(img_path)
                face, prob = mtcnn(img, return_prob=True)
                emb = resnet(face.unsqueeze(0)).detach()

                dist_list = []
                for idx, emb_db in enumerate(embedding_list):
                    dist = torch.dist(emb, emb_db).item()
                    dist_list.append(dist)

                idx_min = dist_list.index(min(dist_list))
                predicted_name = name_list[idx_min]
                distance = min(dist_list)

                date, time, day = get_current_datetime()
                writer.writerow({'Predicted Name': predicted_name,
                                 'Date': date, 'Time': time, 'Day': day})


In [11]:
folder_path = '../Images/output_images/testImages/'
data_path = 'data.pt'
output_csv = '../csvFolder/prediction.csv'

In [12]:
predict_and_record(folder_path, data_path, output_csv)

Bhuvanesh
Gokul
Sweeton
Cranio
Siyon
